In [41]:
# Increase max number of columns displayed in output tables
options(repr.matrix.max.cols = 2000)
set.seed(10) # Make sure your ML results are the same
# Calling external libraries for additional functionality
suppressMessages(library(tidyverse))
suppressMessages(library(randomForest))
suppressMessages(library(forcats))
suppressMessages(library(cowplot))
suppressMessages(library(caret))
suppressMessages(library(e1071))
suppressMessages(library(pROC))

cat('Setup complete!')

Setup complete!

In [42]:
# Note: Unicode Transformation Format – 8 (UTF-8) is a standard to encode characters in different languages
cat('Data loading, please wait\n')
diabete_data <- read.csv(file="data/dataset_diabetes/diabetic_data.csv",  encoding="UTF-8", header=TRUE, sep=",")
cat('Data loaded!')

Data loading, please wait
Data loaded!

In [43]:
head(diabete_data)
str(diabete_data)

encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed,readmitted
<int>,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<fct>,<fct>,<int>,<int>,<int>,<int>,<int>,<int>,<fct>,<fct>,<fct>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,?,Pediatrics-Endocrinology,41,0,1,0,0,0,250.83,?,?,1,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO
149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,?,?,59,0,18,0,0,0,276,250.01,255,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,>30
64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,?,?,11,5,13,2,0,1,648,250,V27,6,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO
500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,?,?,44,1,16,0,0,0,8,250.43,403,7,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO
16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,?,?,51,0,8,0,0,0,197,157,250,5,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO
35754,82637451,Caucasian,Male,[50-60),?,2,1,2,3,?,?,31,6,16,0,0,0,414,411,250,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,>30


'data.frame':	101766 obs. of  50 variables:
 $ encounter_id            : int  2278392 149190 64410 500364 16680 35754 55842 63768 12522 15738 ...
 $ patient_nbr             : int  8222157 55629189 86047875 82442376 42519267 82637451 84259809 114882984 48330783 63555939 ...
 $ race                    : Factor w/ 6 levels "?","AfricanAmerican",..: 4 4 2 4 4 4 4 4 4 4 ...
 $ gender                  : Factor w/ 3 levels "Female","Male",..: 1 1 1 2 2 2 2 2 1 1 ...
 $ age                     : Factor w/ 10 levels "[0-10)","[10-20)",..: 1 2 3 4 5 6 7 8 9 10 ...
 $ weight                  : Factor w/ 10 levels ">200","?","[0-25)",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ admission_type_id       : int  6 1 1 1 1 2 3 1 2 3 ...
 $ discharge_disposition_id: int  25 1 1 1 1 1 1 1 1 3 ...
 $ admission_source_id     : int  1 7 7 7 7 2 2 7 4 4 ...
 $ time_in_hospital        : int  1 3 2 2 1 3 4 5 13 12 ...
 $ payer_code              : Factor w/ 18 levels "?","BC","CH",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ medical_speci

In [44]:
diabete_data[diabete_data == '?'] <- NA
diabete_data[diabete_data == 'None'] <- NA

In [45]:
cat('Number of Missing Data for Each Variable:')
sapply(diabete_data, function(x) sum(is.na(x)))

Number of Missing Data for Each Variable:

encounter_id              patient_nbr                     race 
                       0                        0                     2273 
                  gender                      age                   weight 
                       0                        0                    98569 
       admission_type_id discharge_disposition_id      admission_source_id 
                       0                        0                        0 
        time_in_hospital               payer_code        medical_specialty 
                       0                    40256                    49949 
      num_lab_procedures           num_procedures          num_medications 
                       0                        0                        0 
       number_outpatient         number_emergency         number_inpatient 
                       0                        0                        0 
                  diag_1                   diag_2                   diag_3 
                      21                      358                     1423 
        number_diagnoses            max_glu_serum                A1Cresult 
                       0                    96420                    84748 
               metformin              repaglinide              nateglinide 
                       0                        0                        0 
          chlorpropamide              glimepiride            acetohexamide 
                       0                        0                        0 
               glipizide                glyburide              tolbutamide 
                       0                        0                        0 
            pioglitazone            rosiglitazone                 acarbose 
                       0                        0                        0 
                miglitol             troglitazone               tolazamide 
                       0                        0                        0 
                 examide              citoglipton                  insulin 
                       0                        0                        0 
     glyburide.metformin      glipizide.metformin glimepiride.pioglitazone 
                       0                        0                        0 
 metformin.rosiglitazone   metformin.pioglitazone                   change 
                       0                        0                        0 
             diabetesMed               readmitted 
                       0                        0

In [46]:
table(diabete_data$admission_source_id, exclude = NULL)


    1     2     3     4     5     6     7     8     9    10    11    13    14 
29565  1104   187  3187   855  2264 57494    16   125     8     2     1     2 
   17    20    22    25 
 6781   161    12     2 

In [47]:
# Missing values
diabete_data$admission_source_id[diabete_data$admission_source_id == 9] <- NA
diabete_data$admission_source_id[diabete_data$admission_source_id == 15] <- NA
diabete_data$admission_source_id[diabete_data$admission_source_id == 17] <- NA
diabete_data$admission_source_id[diabete_data$admission_source_id == 20] <- NA
diabete_data$admission_source_id[diabete_data$admission_source_id == 21] <- NA

In [48]:
# convert less common values to 'Other'
diabete_data$admission_source_id[diabete_data$admission_source_id==8 
                                | diabete_data$admission_source_id==10
                                | diabete_data$admission_source_id==11
                                | diabete_data$admission_source_id==13
                                | diabete_data$admission_source_id==14
                                | diabete_data$admission_source_id==22
                                | diabete_data$admission_source_id==25] <- 0

In [49]:
table(diabete_data$admission_source_id, exclude = NULL)


    0     1     2     3     4     5     6     7  <NA> 
   43 29565  1104   187  3187   855  2264 57494  7067 

In [50]:
# Create 'admission_source'
diabete_data <- mutate(diabete_data, admission_source = ifelse(admission_source_id == 0, 'Other', 
                                            ifelse(admission_source_id == 1, 'Physician_Referral',
                                            ifelse(admission_source_id == 2, 'Clinic_Referral',
                                            ifelse(admission_source_id == 3, 'HMO_Referral',
                                            ifelse(admission_source_id == 4, 'Transfer_from_Hospital',
                                            ifelse(admission_source_id == 5, 'Transfer_from_SNF',
                                            ifelse(admission_source_id == 6, 'Transfer_from_Healthcare_Facility',
                                            ifelse(admission_source_id == 7, 'Emergency_Room', NA)))))))))

In [51]:
head(diabete_data[c('admission_source_id', 'admission_source')])

admission_source_id,admission_source
<dbl>,<chr>
1,Physician_Referral
7,Emergency_Room
7,Emergency_Room
7,Emergency_Room
7,Emergency_Room
2,Clinic_Referral


In [52]:
table(diabete_data$admission_type_id, exclude = NULL)


    1     2     3     4     5     6     7     8 
53990 18480 18869    10  4785  5291    21   320 

In [53]:
# Missing values
diabete_data$admission_type_id[diabete_data$admission_type_id == 5] <- NA # Not Available
diabete_data$admission_type_id[diabete_data$admission_type_id == 6] <- NA # NULL
diabete_data$admission_type_id[diabete_data$admission_type_id == 8] <- NA # Not Mapped

In [54]:
# convert less common values to 'Other'
diabete_data$admission_type_id[diabete_data$admission_type_id==4 
                                | diabete_data$admission_type_id==7] <- 0

In [55]:
table(diabete_data$admission_type_id, exclude = NULL)


    0     1     2     3  <NA> 
   31 53990 18480 18869 10396 

In [56]:
# Create 'admission_type'
diabete_data <- mutate(diabete_data, admission_type = ifelse(admission_type_id == 0, 'Other', 
                                            ifelse(admission_type_id == 1, 'Emergency',
                                            ifelse(admission_type_id == 2, 'Urgent',
                                            ifelse(admission_type_id == 3, 'Elective', NA)))))

In [58]:
head(diabete_data[c('admission_type_id', 'admission_type')])

admission_type_id,admission_type
<dbl>,<chr>
NA,NA
1,Emergency
1,Emergency
1,Emergency
1,Emergency
2,Urgent


In [59]:
table(diabete_data$discharge_disposition_id, exclude = NULL)


    1     2     3     4     5     6     7     8     9    10    11    12    13 
60234  2128 13954   815  1184 12902   623   108    21     6  1642     3   399 
   14    15    16    17    18    19    20    22    23    24    25    27    28 
  372    63    11    14  3691     8     2  1993   412    48   989     5   139 

In [60]:
# Missing values
diabete_data$discharge_disposition_id[diabete_data$discharge_disposition_id == 18] <- NA # NULL
diabete_data$discharge_disposition_id[diabete_data$discharge_disposition_id == 25] <- NA # Not Mapped
diabete_data$discharge_disposition_id[diabete_data$discharge_disposition_id == 26] <- NA # Unknown/Invalid


In [62]:
# convert less common values to 'Other'
diabete_data$discharge_disposition_id[diabete_data$discharge_disposition_id==9 
                                     | diabete_data$discharge_disposition_id==10
                                     | diabete_data$discharge_disposition_id==12
                                     | diabete_data$discharge_disposition_id==15
                                     | diabete_data$discharge_disposition_id==16
                                     | diabete_data$discharge_disposition_id==17
                                     | diabete_data$discharge_disposition_id==19
                                     | diabete_data$discharge_disposition_id==20
                                     | diabete_data$discharge_disposition_id==24
                                     | diabete_data$discharge_disposition_id==27] <- 0

In [63]:
table(diabete_data$discharge_disposition_id, exclude = NULL)


    0     1     2     3     4     5     6     7     8    11    13    14    22 
  181 60234  2128 13954   815  1184 12902   623   108  1642   399   372  1993 
   23    28  <NA> 
  412   139  4680 

In [65]:
# Create 'discharge_disposition'
diabete_data <- mutate(diabete_data, discharge_disposition = ifelse(discharge_disposition_id == 0, 'Other', 
                                            ifelse(discharge_disposition_id == 1, 'Home',
                                            ifelse(discharge_disposition_id == 2, 'Short_term_hospital',
                                            ifelse(discharge_disposition_id == 3, 'SNF',
                                            ifelse(discharge_disposition_id == 4, 'ICF',
                                            ifelse(discharge_disposition_id == 5, 'another_type_of_inpatient_care_institution',
                                            ifelse(discharge_disposition_id == 6, 'Home_with_home_health_service',
                                            ifelse(discharge_disposition_id == 7, 'Left_AMA',
                                            ifelse(discharge_disposition_id == 8, 'home_under_care_of_home_IV_provider',
                                            ifelse(discharge_disposition_id == 11, 'Expired',
                                            ifelse(discharge_disposition_id == 13, 'Hospice/Home',
                                            ifelse(discharge_disposition_id == 14, 'Hospice/Medical_facility',
                                            ifelse(discharge_disposition_id == 22, 'Another_rehab_fac_including_rehab_units_of_a_hospital',
                                            ifelse(discharge_disposition_id == 23, 'Long_term_care_hospital',
                                            ifelse(discharge_disposition_id == 28, 'Psychiatric_hospital_of_psychiatric_distinct_part_unit_of_a_hospital',
                                                   NA))))))))))))))))

In [66]:
head(diabete_data[c('discharge_disposition_id', 'discharge_disposition')])

discharge_disposition_id,discharge_disposition
<dbl>,<chr>
NA,NA
1,Home
1,Home
1,Home
1,Home
1,Home


In [69]:
diabete_data$admission_source <- as.factor(diabete_data$admission_source)
diabete_data$admission_type <- as.factor(diabete_data$admission_type)
diabete_data$discharge_disposition <- as.factor(diabete_data$discharge_disposition)

cat('Conversion complete!')

Conversion complete!

In [71]:
summary(diabete_data[,-1])

  patient_nbr                     race                   gender     
 Min.   :      135   ?              :    0   Female         :54708  
 1st Qu.: 23413221   AfricanAmerican:19210   Male           :47055  
 Median : 45505143   Asian          :  641   Unknown/Invalid:    3  
 Mean   : 54330401   Caucasian      :76099                          
 3rd Qu.: 87545950   Hispanic       : 2037                          
 Max.   :189502619   Other          : 1506                          
                     NA's           : 2273                          
      age              weight      admission_type_id discharge_disposition_id
 [70-80):26068   [75-100) : 1336   Min.   :0.000     Min.   : 0.000          
 [60-70):22483   [50-75)  :  897   1st Qu.:1.000     1st Qu.: 1.000          
 [50-60):17256   [100-125):  625   Median :1.000     Median : 1.000          
 [80-90):17197   [125-150):  145   Mean   :1.615     Mean   : 2.924          
 [40-50): 9685   [25-50)  :   97   3rd Qu.:2.000     3rd Q